# Challenge- News Analysis

![](https://static01.nyt.com/newsgraphics/images/icons/defaultPromoCrop.png)

Picture from [The New York Times](www.nytimes.com)

---

In [12]:
import spacy
from spacy import displacy

from collections import Counter

from bs4 import BeautifulSoup
import requests
import re

nlp = spacy.load("en_core_web_md")

# TODO : load statistical models

# 1. Create a scrapper

### voir le cours Data Collection/03-Web Scrapping

We are going to scrape the following webpages : 
- https://www.nytimes.com/2019/08/06/upshot/china-us-trade-war-currency-markets.html
- https://www.lesechos.fr/economie-france/social/deliveroo-dans-le-collimateur-de-la-justice-espagnole-1122616

Write a function `url_to_string(url)` that returns the text content of the given webpage.

In [37]:
#from requests_html import HTMLSession
#session = HTMLSession()

# fonction de la correction
def url_to_string(url):
    # Send a GET request to a regular URL (not an API this time)
    res = requests.get(url)
    print(res.status_code) ### error 403 !!!
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

## NOk avec NYTimes ; OK avec LesEchos
# remplacer l'url eng par : https://www.npr.org/2024/05/31/g-s1-2024/trump-trial-verdict-analysis
url_nytimes = "https://www.npr.org/2024/05/31/g-s1-2024/trump-trial-verdict-analysis"
text_nytimes = url_to_string(url_nytimes)
print(f'npr: {text_nytimes}')

url_lesechos = "https://www.lesechos.fr/economie-france/social/deliveroo-dans-le-collimateur-de-la-justice-espagnole-1122616"
text_lesechos = url_to_string(url_lesechos)
print(f'lesechos: {text_lesechos}')

200
npr:  4 political takeaways from Trump's historic felony conviction : NPR Accessibility links  Skip to main content Keyboard shortcuts for audio player                     Open Navigation Menu                  Newsletters NPR Shop                     Close Navigation Menu Home News Expand/collapse submenu for News National World Politics Business Health Science Climate Race Culture Expand/collapse submenu for Culture Books Movies Television Pop Culture Food Art & Design  Performing Arts Life Kit Gaming Music Expand/collapse submenu for Music         Tiny Desk              Hip-Hop 50              All Songs Considered              Music Features              Live Sessions      Podcasts & Shows Expand/collapse submenu for Podcasts & Shows Daily                                     Morning Edition                                                                      Weekend Edition Saturday                                                                      Weekend Edition Sunday       

In [29]:
def labels_entries(doc):
    entries = []
    labels  = []
    for ent in doc.ents:
        print(ent.text, ent.start_char, ent.end_char, ent.label_, ' - ', spacy.explain(ent.label_))
        entries.append(ent)
        labels.append(ent.label_)
    return entries, labels

# 2. Analyse English News with NER

In [39]:
nlp_en = spacy.load("en_core_web_md")

In [43]:
# TODO: load entities
# create Doc object
docs_en = nlp_en(text_nytimes)

entries_en, labels_en = labels_entries(docs_en)
entries_en


4 1 2 CARDINAL  -  Numerals that do not fall under another type
Trump 28 33 ORG  -  Companies, agencies, institutions, etc.
NPR Accessibility 65 82 ORG  -  Companies, agencies, institutions, etc.
Keyboard 111 119 ORG  -  Companies, agencies, institutions, etc.
NPR Shop 217 225 ORG  -  Companies, agencies, institutions, etc.
Navigation Menu Home News Expand 252 284 ORG  -  Companies, agencies, institutions, etc.
News National World Politics Business Health Science Climate Race Culture Expand 306 386 ORG  -  Companies, agencies, institutions, etc.
Culture Books Movies Television Pop Culture Food Art & Design   408 471 ORG  -  Companies, agencies, institutions, etc.
Music         Tiny Desk 537 560 ORG  -  Companies, agencies, institutions, etc.
Music Features              Live Sessions      Podcasts & Shows Expand 632 702 WORK_OF_ART  -  Titles of books, songs, etc.
Podcasts & Shows Daily                                     Morning Edition                                                  

[4,
 Trump,
 NPR Accessibility,
 Keyboard,
 NPR Shop,
 Navigation Menu Home News Expand,
 News National World Politics Business Health Science Climate Race Culture Expand,
 Culture Books Movies Television Pop Culture Food Art & Design  ,
 Music         Tiny Desk,
 Music Features              Live Sessions      Podcasts & Shows Expand,
 Podcasts & Shows Daily                                     Morning Edition                                                                      Weekend Edition,
 Saturday                                                                      Weekend Edition,
 Sunday,
 Rachel Martin,
 NPR Shop         Tiny Desk,
 Music Features              Live Sessions,
 NPR Diversity Support Careers Press Ethics,
 4,
 Trump,
 Trump,
 Republican,
 4,
 Donald Trump,
 May 31,
 20245:00 AM ET,
 Domenico Montanaro,
 Donald Trump,
 Manhattan Criminal Court,
 New York,
 May 20, 2024,
 Trump,
 34,
 Thursday,
 Dave Sanders,
 The New York Times,
 AP,
 Pool,
 Dave Sanders,
 The New

In [44]:
docs_en.ents

(4,
 Trump,
 NPR Accessibility,
 Keyboard,
 NPR Shop,
 Navigation Menu Home News Expand,
 News National World Politics Business Health Science Climate Race Culture Expand,
 Culture Books Movies Television Pop Culture Food Art & Design  ,
 Music         Tiny Desk,
 Music Features              Live Sessions      Podcasts & Shows Expand,
 Podcasts & Shows Daily                                     Morning Edition                                                                      Weekend Edition,
 Saturday                                                                      Weekend Edition,
 Sunday,
 Rachel Martin,
 NPR Shop         Tiny Desk,
 Music Features              Live Sessions,
 NPR Diversity Support Careers Press Ethics,
 4,
 Trump,
 Trump,
 Republican,
 4,
 Donald Trump,
 May 31,
 20245:00 AM ET,
 Domenico Montanaro,
 Donald Trump,
 Manhattan Criminal Court,
 New York,
 May 20, 2024,
 Trump,
 34,
 Thursday,
 Dave Sanders,
 The New York Times,
 AP,
 Pool,
 Dave Sanders,
 The New

In [46]:
# TODO: labels count
label_types = set(labels_en)
print(label_types)
print(len(label_types))

# cours
labels = [x.label_ for x in docs_en.ents]
Counter(labels)

{'ORG', 'WORK_OF_ART', 'DATE', 'PERSON', 'GPE', 'ORDINAL', 'FAC', 'TIME', 'CARDINAL', 'NORP'}
10


Counter({'CARDINAL': 12,
         'ORG': 40,
         'WORK_OF_ART': 2,
         'DATE': 26,
         'PERSON': 43,
         'NORP': 15,
         'TIME': 1,
         'GPE': 11,
         'ORDINAL': 4,
         'FAC': 1})

In [47]:
# TODO: most common items
# cours
items = [x.text for x in docs_en.ents]
Counter(items).most_common(3)


[('Trump', 30), ('Biden', 14), ('Donald Trump', 5)]

In [59]:
# dependencies
for token in docs_en:
    print(token.text, token.dep_, token.head.text)

  dep 4
4 nummod takeaways
political amod takeaways
takeaways nsubj Shows
from prep takeaways
Trump poss conviction
's case Trump
historic amod conviction
felony compound conviction
conviction pobj from
: punct takeaways
NPR compound Accessibility
Accessibility compound links
links appos takeaways
  dep links
Skip appos takeaways
to prep Skip
main amod shortcuts
content compound Keyboard
Keyboard compound shortcuts
shortcuts pobj to
for prep shortcuts
audio amod Newsletters
player compound Newsletters
                     dep player
Open compound Newsletters
Navigation compound Newsletters
Menu compound Newsletters
                  dep Menu
Newsletters compound submenu
NPR compound Close
Shop compound Close
                     dep Shop
Close compound submenu
Navigation compound Menu
Menu compound News
Home compound News
News compound submenu
Expand nmod collapse
/ punct collapse
collapse compound submenu
submenu pobj for
for prep submenu
News compound submenu
National compound submen

In [49]:
# TODO: NER on the sentence
# cours
sentences = [x for x in docs_en.sents]
print(sentences[13])

displacy.render(nlp_en(str(sentences[13])), style='dep', jupyter = True, options = {'distance': 120})

Donald Trump is still going to be the Republican nominee.


# 3. Analyse French News with NER

In [51]:
nlp_fr = spacy.load("fr_core_news_md")

In [52]:
# TODO: load entities
docs_fr = nlp_fr(text_lesechos)
entries_fr, labels_fr = labels_entries(docs_fr)

Deliveroo 1 10 PER  -  Named person or family.
Les Echos 57 66 MISC  -  Miscellaneous entities, e.g. events, nationalities, products or works of art
InvestirRechercheEn continuLe journalNewslettersMes articlesMes 86 149 LOC  -  Non-GPE locations, mountain ranges, bodies of water
secteursPodcastsInfographiesVidéosThemaThèmeClairSe connecterS’ 150 213 PER  -  Named person or family.
Valence 484 491 LOC  -  Non-GPE locations, mountain ranges, bodies of water
Madrid 497 503 LOC  -  Non-GPE locations, mountain ranges, bodies of water
RestaurationLa 689 703 PER  -  Named person or family.
Deliveroo 737 746 PER  -  Named person or family.
Sécurité sociale 764 780 ORG  -  Companies, agencies, institutions, etc.
Shutterstock)Par 782 798 MISC  -  Miscellaneous entities, e.g. events, nationalities, products or works of art
Cécile ThibaudPublié 799 819 PER  -  Named person or family.
Deliveroo 899 908 LOC  -  Non-GPE locations, mountain ranges, bodies of water
Sécurité sociale 1075 1091 ORG  -  Co

In [53]:
# TODO: labels count
label_types = set(labels_fr)
print(label_types)
print(len(label_types))

# cours
labels = [x.label_ for x in docs_fr.ents]
Counter(labels)

{'LOC', 'PER', 'ORG', 'MISC'}
4


Counter({'PER': 15, 'MISC': 16, 'LOC': 17, 'ORG': 10})

In [54]:
# TODO: most common items
items = [x.text for x in docs_fr.ents]
Counter(items).most_common(3)

[('Deliveroo', 8), ('Madrid', 4), ('Valence', 3)]

In [58]:
# TODO: dependencies on a given sentence
sentence = entries_fr[0]
print(sentence)
#Deliveroo dans le collimateur de la justice espagnole | Les Echos Voir la bourse

for token in docs_fr:
    print(token.text, token.dep_, token.head.text)

Deliveroo
  dep  
Deliveroo ROOT Deliveroo
dans case collimateur
le det collimateur
collimateur nmod Deliveroo
de case justice
la det justice
justice nmod collimateur
espagnole amod justice
| punct Deliveroo
Les det Echos
Echos ROOT Echos
Voir ROOT Voir
la det bourse
bourse obj Voir
sur case InvestirRechercheEn
InvestirRechercheEn nmod bourse
continuLe acl bourse
journalNewslettersMes nmod bourse
articlesMes amod journalNewslettersMes
secteursPodcastsInfographiesVidéosThemaThèmeClairSe amod journalNewslettersMes
connecterS’ nmod bourse
abonnerÀ acl bourse
la det uneIdéesÉconomiePolitiqueEuropéennes
uneIdéesÉconomiePolitiqueEuropéennes obj abonnerÀ
2024EntreprisesFinance obj Voir
- punct MarchésBourseMondeTech
MarchésBourseMondeTech ROOT MarchésBourseMondeTech
- ROOT -
MédiasStart ROOT MédiasStart
- ROOT -
upRégionsPatrimoineLe ROOT upRégionsPatrimoineLe
Mag ROOT Mag
W ROOT W
- ROOT -
ESocialBudget ROOT ESocialBudget
FiscalitéConjonctureSocialPublicitéDeliveroo acl ESocialBudget
dans ca

In [57]:
# TODO: NER on the sentence
sentences = [x for x in docs_fr.sents]
print(sentences[13])

displacy.render(nlp_fr(str(sentences[13])), style='dep', jupyter = True, options = {'distance': 120})

dans le collimateur de la justice espagnoleDeux jugements récents rendus à Valence et à Madrid reconnaissent les livreurs à vélo comme salariés de la plate-forme et lui réclament les cotisations sociales impayées.
